*Is there anything more useless or less usefull than algebra?* \
*- Billy Connoly*

Linear Algebra is the branch of mathematics that deals with *vector spaces* 

# Vectors

Abstractly, *vectors* are objects that can be added together to form new vectors and that can be multiplied by *scalars* (i.e, numbers, also to form new vectors).

Concretely (for us), vectors are points in some finite-dimensional space. Although we might not think of our data as vectors, they are often a useful way to represent numeric data.

For example, if you have the heights, weights, and ages of a large number of people, you can treat your data as three-dimensional vectors [height, weight, age].
If you are teaching a class with four exams, you can treat student grades as four-dimensional vectors [exam1, exam2, exam3, exam4]

The simplest form-scratch approach is to represent vectors as lists of numbers. A list of three numbers corresponds to a vector in three-dimensional space, and vice versa.


We will accomplish this with a type alias that says a Vector is just a list of floats:

In [1]:
from typing import List

vector = List[float]

height_weight_age = [70, # inches
                     170, # pounds
                     40] # years

grades = [95, # exam1
          80, # exam2
          75, # exam3
          62,] # exam4


We will also want to perform *arithmetic* on vectors. Because Python `list` aren't vectors (and hence provide no facilities for vector arithmetic), 
we will need to build these arithmetic tools ourselves.

To begin with, we will frequently need to add two vectors. Vectors add *componentwise*.
This means that if two vectors v and w are the same length, their sum is just the vector whose first element is v[0] + w[0], who's second element is v[1] + w[1] and so on.
(If they are not the same length then we can not add them).

For example, adding the vectors [1, 2] and [2, 1] results in [1 + 2,  2 + 1] or [3, 3].

We can easily do this by zipping them together and using list comprehension to add the corresponding elements:
    

In [2]:
def add(v: vector, w: vector) -> vector:
    """ Adds corresponding elements"""
    assert len(v) == len(w), "Vectors must be the same length"
    return [v_i + w_i for v_i, w_i in zip(v, w)]
assert add ([1, 2, 3], [4, 5, 6]) == [5, 7, 9]

In [3]:
add([1, 2, 3], [4, 5, 6])

[5, 7, 9]

In [4]:
def subtract(v: vector, w: vector) -> vector:
    """Subtracts corresponding elements"""
    assert len(v) == len(w), "vectors must be the same length"
    return [v_i - w_i for v_i, w_i in zip(v, w)]

assert subtract([5, 7, 9], [4, 5, 6]) == [1, 2, 3]

In [5]:
subtract([5, 7, 9], [4, 5, 6])

[1, 2, 3]

We will sometimes want to componentwise sum a list of vectors - that is - create a new vector whose first element is the sum of all the first elements, whose second element is the sum of all the second elements, and so on:
    

In [6]:
def vector_sum(vectors: list[vector]) -> vector:
    """Sums all corresponding elements"""
    # Check that vectors is not empty
    assert vectors, "no vectors provided"
    
    # Check the vectors are all the same size
    num_elements = len(vectors[0])
    assert all(len(v) == num_elements for v in vectors), "different sizes"
    
    # the i-th element of the result is the sum of every vector[i]
    return [sum(vector[i] for vector in vectors)
           for i in range(num_elements)]

assert vector_sum([[1, 2], [3, 4], [5, 6], [7, 8]]) == [16, 20]

In [7]:
vector_sum([[1, 2], [3, 4], [5, 6], [7, 8]])

[16, 20]

We will also need to be able to multiply a vector by a scalar, which we do simply by multiplying each element of the vector by that number:

In [8]:
def scalar_multiply(c: float, v: vector) -> vector:
    """Multiplies every element by c"""
    return [c * v_i for v_i in v]

assert scalar_multiply(2, [1, 2, 3]) == [2, 4, 6]

In [9]:
scalar_multiply(2, [1, 2, 3])

[2, 4, 6]

This allows us to compute the componentwise means of a list of (same-sized) vectors:
    

In [10]:
def vector_mean(vectors: List[vector]) -> vector:
    """Computes the element-wise average"""
    n = len(vectors)
    return scalar_multiply(1/n, vector_sum(vectors))

assert vector_mean([[1, 2], [3, 4], [5, 6]]) == [3, 4]

In [11]:
vector_mean([[1, 2], [3, 4], [5, 6]])

[3.0, 4.0]

A less obvious tool is the *dot product*. The dot product of two vectors is the sum of their componentwise products:

In [12]:
def dot(v: vector, w: vector) -> float:
    """Computes v_i * w_i + ... + v_n * w_n"""
    assert len(v) == len(w), "vectors must be same length"
    
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

assert dot([1, 2, 3], [4, 5, 6]) == 32 # 1 * 4 + 2 * 5 + 3 * 6

In [13]:
dot([1, 2, 3], [4, 5, 6])

32

In [14]:
# dot([1, 2, 3], [4, 5 ])

If w has magnitude 1, the dot product measures how far the vector v extends in the w direction.

For example, if w = [1, 0], then `dot(v, w)` is just the first component of v.

Another way of saying this is that it's the length of the vector you would get if you projected v onto w.

Using this, it's easy to compute a vector's sum of squares:

In [15]:
def sum_of_squares(v:vector) -> float:
    """Returns v_1 * v_1 + ... + v_n * v_n"""
    return dot(v, v)

In [16]:
sum_of_squares([1, 2, 3])

14

which we can use to compute it's *magnitude* (or length):

In [17]:
import math

In [18]:
def magnitude(v: vector) -> float:
    """Returns the magnitude (or length) of v """
    return math.sqrt(sum_of_squares(v))

In [19]:
magnitude([3, 4])

5.0

We now have all the pieces we need to compute the distance between two vectors, defined as:
    
    ((v1 - w1)^2 + ... + (vn -wn)^2)^-2

In code:

In [20]:
def squared_distance(v:vector, w: vector) -> float:
    """Computes (v_1 - w_1) ** 2 + ... + (v_n - w_n) ** 2"""
    return sum_of_squares(subtract(v, w))

def distance(v: vector, w: vector) -> float:
    """Computes the distance between v and w"""
    return math.sqrt(squared_distance(v, w))

🔑 **Note:** 🔑 Using lists as vectors is great for exposition but terrible for performance.
In production code, you would want to use the NumPy library, which includes a high-performance array class with all sorts of arithmetic operations included.

## Matrices

A *matrix* is a two-dimensional collection of numbers. We will represent matrices as lists of lists, with each inner list having the same size and representing a *row* of the matrix. If **A** is a matrix, then **A[i][j]** is the element in the *i*th row and the *j*th column.
Per mathematical convention, we will frequently use capital letters to represent matrices. For example:

In [21]:
# Another type alias
Matrix = list[list[float]]

A = [[1, 2, 3], # A has 2 rows and 3 columns
     [4, 5, 6]]

B = [[1, 2],  # B has 3 rows and 2 columns
     [3, 4],
     [5, 6]]

🔑 **Note:** 🔑 In mathematics, you would usually name the first row of the matrix "row 1" and the first column "column 1." Because we are representing matrices with Python `lists`, which are zero-indexed, we will call the first row of a matrix "row 0" and the first column "column 0."

Given this list-of-lists representation, the matrix A has `len`(A) rows and `len`(A[0]) columns, which we consider its shape: 

In [22]:
from typing import Tuple

def  shape(A: Matrix) -> Tuple[int, int]:
    """Returns (# of rows of A, # of columns of A )"""
    num_rows = len(A)
    num_cols = len(A[0]) if A else 0  # number of elements in first row
    return num_rows, num_cols

In [23]:
shape([[1, 2, 3], [4, 5, 6]])

(2, 3)

In [24]:
len(A)

2

In [25]:
len(A[0])

3

In [26]:
A[0]

[1, 2, 3]

In [27]:
shape([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

(3, 3)

If a matrix has *n* rows and *k* columns, we will refer to it as an *n* x *k* *matrix*. We can (and sometimes will) think of each row of an *n* x *k* matrix as a vector of length *k*, and each column as a vector of length *n*:

In [28]:
def get_row(A: Matrix, i: int) -> vector:
    """Returns the i-th row of A (as a vector)"""
    return A[i]                # A[i] is already the ith row


def get_column(A: Matrix, j: int) -> vector:
    """Returns the j-th column of A (as a vector)"""
    return [A_i[j]               # jth element of row A_i
            for A_i in A]        # for each row A_i             


We'll also want to be able to create a matrix given its shape and a function for generating its elements. We can do this using a nested list comprehension:

In [29]:
from typing import Callable

def make_matrix(num_rows: int,
                num_cols: int,
                entry_fn: Callable[[int, int], float]) -> Matrix:
    """Returns a num_rows x num_cols matrix
        whose (i,j)-th entry is entry_fn(i, j)
    """
    return [[entry_fn(i, j)              # given i, create a list
            for j in range(num_cols)]    # [entry_fn(i,0), ...]
           for i in range(num_rows)]     # create one list for each i

Given this function, you could make a 5x5 *identity matrix* (with 1s on the diagonal and 0s elsewhere) like so:

In [30]:
def identity_matrix(n: int) -> Matrix:
    """Returns the n x n identity matrix"""
    return make_matrix(n, n, lambda i, j: 1 if i == j else 0)

In [31]:
identity_matrix(5)

[[1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1]]

Matrices will be important for several reasons.

First, we can use a matrix to represent a dataset consisting of multiple vectors, simply by considering each vector as a row of the Matrix. For example, if you had the heights, weights, and ages of 1,000 people, you could put them in a 1,000 x 3 matrix:

In [32]:
data = [[70, 170, 40],
        [65, 120, 26],
        [77, 250, 19],
        # ...
       ]